<a href="https://colab.research.google.com/github/NARAEIM/-/blob/main/n213a_regularized_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# 🌱 **기본 과제**

이번에는 노트에서 다루지 않은 새로운 데이터인 [Melbourne Housing Market](https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market#:~:text=calendar_view_week-,MELBOURNE_HOUSE_PRICES_LESS,-.csv)을 사용하여 정규화를 연습해 보겠습니다.

In [1]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Part 1. Preprocessing**

### **1-1. 데이터 전처리**

- 데이터셋을 불러온 후 전처리를 수행합니다.
  - 모델링을 하기 전에 아래 항목을 drop 해주세요.
    - 특성
      - 날짜 특성
      - 범주형 데이터 중에서 카디널리티가 50이 넘는 특성
    - 행
      - Price 변수 기준 결측치 행
      - 중복되는 행

#### **Q1. 전처리 후 데이터의 shape을 제출하세요.**

In [2]:
# 데이터셋을 불러옵니다. 

import pandas as pd

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')

In [3]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63023 entries, 0 to 63022
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         63023 non-null  object 
 1   Address        63023 non-null  object 
 2   Rooms          63023 non-null  int64  
 3   Type           63023 non-null  object 
 4   Price          48433 non-null  float64
 5   Method         63023 non-null  object 
 6   SellerG        63023 non-null  object 
 7   Date           63023 non-null  object 
 8   Postcode       63023 non-null  int64  
 9   Regionname     63023 non-null  object 
 10  Propertycount  63023 non-null  int64  
 11  Distance       63023 non-null  float64
 12  CouncilArea    63023 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.3+ MB


In [5]:
## 이 곳에서 과제를 진행해주세요.

# 날짜 특성 삭제
df= df.drop(columns='Date')
df.shape

(63023, 12)

In [6]:
df.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Postcode           int64
Regionname        object
Propertycount      int64
Distance         float64
CouncilArea       object
dtype: object

In [7]:
df.nunique()


Suburb             380
Address          57754
Rooms               14
Type                 3
Price             3417
Method               9
SellerG            476
Postcode           225
Regionname           8
Propertycount      368
Distance           180
CouncilArea         34
dtype: int64

In [8]:
df2=df[df.columns.difference(['Rooms','Price','Distance'])]
df2=df2.drop(columns=df2[['Address','SellerG', 'Suburb']])
df2.shape

(63023, 6)

In [9]:
df_clean = df.drop(columns = ['Address', 'SellerG', 'Suburb'])
df_clean.shape

(63023, 9)

In [10]:
# Price열에 있는 결측치행 제거
df_clean = df_clean.dropna(subset=['Price'], axis=0)


In [11]:
# 중복행 제거
df_clean = df_clean.drop_duplicates()
df_clean.shape

(42515, 9)

### **1-2. OneHotEncoder**
- 데이터 분리
  - 특성과 타겟을 분리해주세요.
  - `train_test_split()`을 사용하여 전체 데이터를 훈련과 테스트 데이터로 나누어주세요. 
    - 훈련과 테스트 데이터는 8:2의 비로 나눕니다.
    - `random_state=2`로 설정합니다. 
  - `train_test_split()`을 사용하여 훈련 데이터를 훈련과 검증 데이터로 나누어주세요.
    - 훈련과 검증 데이터는 8:2의 비로 나눕니다.
    - `random_state=2`로 설정합니다.
- OneHotEncoder
  - `category_encoders` 라이브러리를 사용해서 원핫인코딩을 수행하세요.

In [12]:
df_clean.columns

Index(['Rooms', 'Type', 'Price', 'Method', 'Postcode', 'Regionname',
       'Propertycount', 'Distance', 'CouncilArea'],
      dtype='object')

In [13]:
## 이 곳에서 과제를 진행해주세요.

# 특성과 타겟을 분리
target = 'Price'
features = ['Rooms', 'Type', 'Method', 'Postcode', 'Regionname', 'Propertycount', 'Distance', 'CouncilArea']

In [14]:
# train, test 데이터 나누기 

from sklearn.model_selection import train_test_split
X = df_clean.drop('Price', axis=1)
y = df_clean['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [15]:
# train, test 데이터가 잘 나눠졌는지 확인

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((34012, 8), (8503, 8), (34012,), (8503,))

In [16]:
# train_test_split()을 사용하여 훈련 데이터를 훈련과 검증 데이터로 나누기

X2_train, X2_val, y2_train, y2_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2)

X2_train.shape, X2_val.shape, y2_train.shape, y2_val.shape

((27209, 8), (6803, 8), (27209,), (6803,))

##### **Q2. 훈련/검증/테스트 데이터에 원핫인코딩을 수행한 후 열(column) 개수를 제출하세요.**

In [17]:
## 이 곳에서 과제를 진행해주세요.
from sklearn.preprocessing import StandardScaler

numeric_feats = X_train.dtypes[X_train.dtypes != "object"].index

scaler = StandardScaler()
X_train[numeric_feats] = scaler.fit_transform(X_train[numeric_feats])
X_test[numeric_feats] = scaler.transform(X_test[numeric_feats])

In [18]:
# 수치형 컬럼이 모두 평균 = 0(에 가까운 수), 표준편차 = 1(에 가까운 수)로 변환된 것을 확인할 수 있습니다.

X_train[numeric_feats].describe().T[['mean', 'std']]

,mean,std
Rooms,-9.630724e-17,1.000015
Postcode,3.952566e-16,1.000015
Propertycount,3.029187e-17,1.000015
Distance,-1.128111e-17,1.000015


In [19]:
# One-Hot encoding을 해주겠습니다.
from category_encoders import OneHotEncoder

ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(X_train)
X_test_ohe = ohe.transform(X_test)
X_val_ohe = ohe.transform(X_val)

NameError: ignored

## **Part 2. 정규화 모델**

- 여러 회귀 모델을 만들어서 비교해보세요.
  - 모델을 수행하기 전에 **`baseline`**모델을 만들어주세요.
  - 정규화 모델을 학습하기 전에 **`StandardScaler`**를 사용해서 **표준화**를 수행해주세요.
  - **`ols`**, **`ridge`**, **`lasso`** 회귀모델을 아래와 같은 조건에서 만들고 각각의 회귀계수를 확인해보세요.
    - `LinearRegression()`
    - `Ridge(alpha=1)`
    - `Lasso(alpha=1000, tol=1)`

In [ ]:
## 이 곳에서 과제를 진행해주세요.

### **2-1. Ridge Regression**

##### **Q3. Ridge 회귀모델의 회귀계수 중 `절댓값이 가장 작은 특성`의 이름과 회귀계수를 ``Abc, 123.45``의 형식으로 제출하세요. (반올림하여 소수점 둘째 자리까지)**

In [ ]:
## 이 곳에서 과제를 진행해주세요.

### **2-2. Lasso Regression**

##### **Q4. Lasso 회귀모델의 회귀계수 중 0으로 수렴한 특성의 개수를 제출하세요.**

In [ ]:
## 이 곳에서 과제를 진행해주세요.

### **2-3. 일반화 성능 검증**
-  학습한 정규화 모델 중 검증 세트의 **`MAE`**가 가장 낮은 모델을 선택하고 일반화 성능을 확인하세요.
  -  일반화 성능은 **`Test Set`**를 한 번만 사용해서 확인합니다.
  - 일반화 성능 확인 시 평가지표는 **`MAE`**를 사용해주세요.

##### **Q5. 정규화 모델 중 검증 세트의 MAE가 가장 낮은 모델로 일반화 성능을 확인하여 점수를 제출해주세요. (반올림하여 소수점 둘째 자리까지)** 

In [ ]:
## 이 곳에서 과제를 진행해주세요.

# 🥇 **도전 과제**

- RidgeCV와 LassoCV를 사용해서 최적의 알파값을 찾는 과정을 수행하고 성능을 개선해보세요.

- ElasticNet 정규화 모델에 대해서 추가로 학습해보세요. 

- Lasso 와 Ridge에 사용되는 L1, L2 정규화에 대해 알아보고 논의해보세요.
